In [1]:
%%capture
! pip install pyarrow

In [2]:
import os
import pandas as pd
import numpy as np
from typing import List, Tuple

In [3]:
# df = pd.read_stata('data/extracted/randhrs1992_2022v1.dta', convert_categoricals=False)
# df.to_parquet('data/extracted/randhrs1992_2022v1.parquet')
# df.columns = df.columns.str.upper()
df = pd.read_parquet('data/extracted/randhrs1992_2022v1.parquet')
df.head()

,HHIDPN,S1HHIDPN,R1MSTAT,R1MPART,S1BMONTH,S1BYEAR,S1BDATE,S1BFLAG,S1COHBYR,S1HRSAMP,...,R8LBSATWLF,R9LBSATWLF,R10LBSATWLF,R11LBSATWLF,R12LBSATWLF,R13LBSATWLF,R14LBSATWLF,R15LBSATWLF,R16LBSATWLF,FILEVER
0,1010,0.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X
1,2010,0.0,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X
2,3010,3020.0,1.0,0.0,9.0,1938.0,-7778.0,0.0,3.0,1.0,...,5.4,NaN,6.4,NaN,NaN,NaN,NaN,NaN,NaN,X
3,3020,3010.0,1.0,0.0,1.0,1936.0,-8752.0,0.0,3.0,1.0,...,4.6,NaN,4.2,NaN,2.4,NaN,NaN,NaN,NaN,X
4,10001010,0.0,8.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.8,NaN,4.8,NaN,NaN,NaN,NaN,NaN,X


In [4]:
WAVE_YEARS = {
    1: 1992,  # HRS cohort entry
    2: 1993,  # AHEAD cohort entry (off-year)
    3: 1994,  # Off-year
    4: 1995,  # Off-year
    5: 1996,  # Biennial pattern begins
    6: 1998,  # CODA/WB cohorts enter
    7: 2000,
    8: 2002,
    9: 2004,  # Early Baby Boomer cohort enters
    10: 2006,
    11: 2008,
    12: 2010, # Mid Baby Boomer cohort enters
    13: 2012,
    14: 2014,
    15: 2016, # Late Baby Boomer cohort enters
    16: 2018,
    17: 2020, # COVID wave
    18: 2022  # Early Generation X cohort enters
}

def get_wave_year(wave: int) -> int | float:
    return WAVE_YEARS.get(wave, np.nan)

def get_variable_name(wave: int, var_base: str) -> str:
    return f"R{wave}{var_base}"

In [5]:
def extract_wave_features(df: pd.DataFrame, wave: int, lags: int = 2):

    data = {}

    # Base identifiers
    data['person_id'] = df['HHIDPN']
    data['wave'] = wave
    data['wave_year'] = WAVE_YEARS[wave]

    # Demographics
    birth_year = pd.to_numeric(df['RABYEAR'], errors='coerce')
    age = WAVE_YEARS[wave] - birth_year # pyright: ignore[reportOperatorIssue]

    data['birth_year'] = birth_year
    data['age'] = age
    data['age_squared'] = age ** 2

    data['female'] = (pd.to_numeric(df['RAGENDER'], errors='coerce') == 2)
    data['white'] = (pd.to_numeric(df['RARACEM'], errors='coerce') == 1)
    data['black'] = (pd.to_numeric(df['RARACEM'], errors='coerce') == 2)
    data['hispanic'] = (pd.to_numeric(df['RAHISPAN'], errors='coerce') == 1)

    data['education_years'] = pd.to_numeric(df['RAEDUC'], errors='coerce')
    data['college_plus'] = (pd.to_numeric(df['RAEDEGRM'], errors='coerce') >= 4)  # pyright: ignore[reportOperatorIssue]

    variables = {
        'self_rated_health': 'SHLT',
        'bmi': 'BMI',
        'weight': 'WEIGHT',
        'height': 'HEIGHT',
        'mobility_limitations': 'MOBILA',
        'large_muscle_limitations': 'LGMUSA',
        'adl_limitations': 'ADL5A',
        'iadl_limitations': 'IADL5A',
        'fine_motor_limitations': 'FINEA',
        'cognition_score': 'COG27',
        'memory_recall': 'TR20',
        'immediate_recall': 'IMRC',
        'delayed_recall': 'DLRC',
        'serial7': 'SER7',
        'depression_score': 'CESD',
        'felt_depressed': 'DEPRES',
        'everything_effort': 'EFFORT',
        'restless_sleep': 'SLEEPR',
        'felt_lonely': 'FLONE',
        'ever_smoked': 'SMOKEV',
        'current_smoker': 'SMOKEN',
        'drinks_per_day': 'DRINKD',
        'drink_days_per_week': 'DRINKN',
        'vigorous_activity': 'VIGACT',
        'marital_status': 'MSTAT'
    }

    # Loop from 0 to lags
    for lag in range(lags + 1):
        w = wave - lag
        if w < 1:
            continue

        suffix = f"_lag{lag}" if lag > 0 else ""

        for name, code in variables.items():
            col = f'R{w}{code}'
            data[f'{name}{suffix}'] = df.get(col)

    features = pd.DataFrame(data)

    return features

In [6]:
def add_temporal_features(df: pd.DataFrame):
    """
    Add temporal features to existing features DataFrame.
    Avoids DataFrame fragmentation by concatenating once.
    """
    
    new_features = {}

    # ========================================
    # BMI TRAJECTORIES
    # ========================================
    new_features['bmi_velocity_2yr'] = (df['bmi'] - df['bmi_lag1']) / 2
    new_features['bmi_velocity_4yr'] = (df['bmi'] - df['bmi_lag2']) / 4
    new_features['bmi_acceleration'] = ((df['bmi'] - df['bmi_lag1']) / 2) - ((df['bmi_lag1'] - df['bmi_lag2']) / 2)
    new_features['weight_change_kg'] = df['weight'] - df['weight_lag1']
    new_features['weight_change_pct'] = (new_features['weight_change_kg'] / df['weight_lag1']) * 100
    new_features['obese'] = (df['bmi'] >= 30).astype(int)
    new_features['overweight'] = ((df['bmi'] >= 25) & (df['bmi'] < 30)).astype(int)
    new_features['normal_weight'] = ((df['bmi'] >= 18.5) & (df['bmi'] < 25)).astype(int)
    new_features['rapid_weight_gain'] = (new_features['bmi_velocity_2yr'] > 1).astype(int)
    new_features['rapid_weight_loss'] = (new_features['bmi_velocity_2yr'] < -1).astype(int)

    # ========================================
    # HEALTH PERCEPTION
    # ========================================
    new_features['health_decline_2yr'] = df['self_rated_health'] - df['self_rated_health_lag1']
    new_features['health_decline_4yr'] = df['self_rated_health'] - df['self_rated_health_lag2']
    new_features['health_worsening'] = (new_features['health_decline_2yr'] > 0).astype(int)
    new_features['health_crash'] = (new_features['health_decline_2yr'] >= 2).astype(int)

    # ========================================
    # FUNCTIONAL DECLINE
    # ========================================
    new_features['mobility_decline_2yr'] = df['mobility_limitations'] - df['mobility_limitations_lag1']
    new_features['mobility_worsening'] = (new_features['mobility_decline_2yr'] > 0).astype(int)
    new_features['new_mobility_problem'] = ((df['mobility_limitations'] > 0) & (df['mobility_limitations_lag1'] == 0)).astype(int)
    new_features['any_mobility_limitation'] = (df['mobility_limitations'] > 0).astype(int)

    new_features['adl_decline_2yr'] = df['adl_limitations'] - df['adl_limitations_lag1']
    new_features['adl_worsening'] = (new_features['adl_decline_2yr'] > 0).astype(int)
    new_features['new_adl_problem'] = ((df['adl_limitations'] > 0) & (df['adl_limitations_lag1'] == 0)).astype(int)
    new_features['any_adl_limitation'] = (df['adl_limitations'] > 0).astype(int)

    new_features['iadl_decline_2yr'] = df['iadl_limitations'] - df['iadl_limitations_lag1']
    new_features['any_iadl_limitation'] = (df['iadl_limitations'] > 0).astype(int)

    # ========================================
    # COGNITIVE DECLINE
    # ========================================
    new_features['cognition_decline_2yr'] = df['cognition_score_lag1'] - df['cognition_score']
    new_features['cognition_decline_4yr'] = df['cognition_score_lag2'] - df['cognition_score']
    new_features['cognition_worsening'] = (new_features['cognition_decline_2yr'] > 0).astype(int)
    new_features['sharp_cognitive_drop'] = (new_features['cognition_decline_2yr'] > 3).astype(int)
    new_features['low_cognition'] = (df['cognition_score'] < 12).astype(int)
    new_features['memory_decline_2yr'] = df['memory_recall_lag1'] - df['memory_recall']
    new_features['memory_worsening'] = (new_features['memory_decline_2yr'] > 0).astype(int)

    # ========================================
    # MENTAL HEALTH
    # ========================================
    new_features['depression_increase_2yr'] = df['depression_score'] - df['depression_score_lag1']
    new_features['depression_worsening'] = (new_features['depression_increase_2yr'] > 0).astype(int)
    new_features['elevated_depression'] = (df['depression_score'] >= 3).astype(int)
    new_features['high_depression'] = (df['depression_score'] >= 4).astype(int)
    new_features['chronic_depression'] = ((df['depression_score'] >= 3) & (df['depression_score_lag1'] >= 3)).astype(int)

    # ========================================
    # HEALTH BEHAVIORS
    # ========================================
    new_features['former_smoker'] = ((df['ever_smoked'] == 1) & (df['current_smoker'] == 0)).astype(int)
    new_features['quit_smoking'] = ((df['current_smoker'] == 0) & (df['current_smoker_lag1'] == 1)).astype(int)
    new_features['sedentary'] = (df['vigorous_activity'] == 0).astype(int)
    new_features['physically_active'] = (df['vigorous_activity'] == 1).astype(int)
    new_features['stopped_activity'] = ((new_features['physically_active'] == 0) & (df['vigorous_activity_lag1'] == 1)).astype(int)
    new_features['drinks_per_week'] = df['drinks_per_day'] * df['drink_days_per_week']
    new_features['heavy_drinking'] = (new_features['drinks_per_week'] > 14).astype(int)

    # ========================================
    # INTERACTIONS
    # ========================================
    new_features['age_x_bmi'] = df['age'] * df['bmi']
    new_features['sedentary_and_obese'] = new_features['sedentary'] * new_features['obese']
    new_features['depression_x_mobility'] = df['depression_score'] * df['mobility_limitations']
    new_features['cognitive_decline_x_age'] = new_features['cognition_decline_2yr'] * df['age']

    # ========================================
    # COMPOSITE SCORES
    # ========================================
    new_features['metabolic_risk_score'] = new_features['obese'] * 2 + new_features['sedentary'] + (df['age'] >= 65).astype(int)
    new_features['frailty_indicators'] = (df['mobility_limitations'] > 0).astype(int) + (df['depression_score'] >= 3).astype(int) + (new_features['weight_change_pct'] < -5).astype(int)

    # Single concat to prevent fragmentation
    df = pd.concat([df, pd.DataFrame(new_features, index=df.index)], axis=1)

    return df


In [7]:
disease_map = {
    'diabetes': 'DIABE',
    'cvd': 'HEARTE', # cardiovascular disease
    'stroke': 'STROK',
    'lung': 'LUNGE',
    'cancer': 'CANCRE',
    'hibp': 'HIBPE', # high blood preasure 
    'arthritis': 'ARTHRE'
}

def create_targets(df: pd.DataFrame, feature_wave: int, outcome_wave: int):

    new_targets = {}
    new_targets['person_id'] = df['HHIDPN']

    for target_name, code in disease_map.items():
        baseline_col = f'R{feature_wave}{code}'
        outcome_col = f'R{outcome_wave}{code}'

        if baseline_col in df.columns and outcome_col in df.columns:
            no_disease_baseline = df[baseline_col] == 0
            develops_disease = df[outcome_col] == 1

            target_values = (no_disease_baseline & develops_disease).astype(float)
            target_values[df[outcome_col].isna()] = np.nan

            new_targets[f'target_{target_name}'] = target_values
            new_targets[f'eligible_{target_name}'] = no_disease_baseline.astype(int)

    # Single DataFrame creation → no fragmentation
    targets = pd.DataFrame(new_targets, index=df.index)

    return targets


In [8]:
def create_dataset(
    df: pd.DataFrame, 
    feature_waves: List[int], 
    prediction_horizon: int = 2, 
    target: str = 'diabetes'
):
    """
    Create a modeling dataset for a specific disease target.
    """
    # Ensure standard naming convention
    if 'hhidpn' in df.columns:
        df.columns = df.columns.str.upper()
    
    all_data = []
    
    for wave in feature_waves:
        outcome_wave = wave + prediction_horizon
        if outcome_wave > 16:
            continue
        
        # 1. Extract wave features and add temporal math (lags, velocity)
        feats = extract_wave_features(df, wave)
        feats = add_temporal_features(feats)
        
        # 2. Create target and eligibility flags for this specific horizon
        targs = create_targets(df, wave, outcome_wave)
        
        # 3. Join features and targets on unique ID
        combined = feats.merge(targs, on='person_id', how='inner')
        all_data.append(combined)
    
    # Combine all selected waves into one master dataframe
    dataset = pd.concat(all_data, ignore_index=True)

    
    # --- FILTERING FOR THE SPECIFIC TARGET ---
    eligible_col = f'eligible_{target}'
    target_col = f'target_{target}'
    
    # Only keep people who did NOT have the disease at the feature wave
    dataset = dataset[dataset[eligible_col] == 1].copy()
    
    # Remove rows where the outcome is unknown (NaN)
    dataset = dataset[pd.notna(dataset[target_col])].copy()

    # Drop rows missing critical predictors to ensure model quality
    dataset = dataset.dropna(subset=('age', 'bmi'))  # pyright: ignore[reportCallIssue]
    
    # --- FEATURE SELECTION ---
    # We must exclude ID, metadata, and ALL target/eligible flags to prevent leakage
    exclude_prefixes = ('target_', 'eligible_')
    metadata_cols = ['person_id', 'wave', 'wave_year']
    
    feature_cols = [c for c in dataset.columns 
                    if not c.startswith(exclude_prefixes) 
                    and c not in metadata_cols]
    
    X = dataset[feature_cols].copy()
    y = dataset[target_col].copy()
    
    return X, y

In [9]:
feature_waves = [5, 6, 7, 8, 9, 10, 11, 12]
prediction_horizon = 2  # 4 years

years = prediction_horizon * 2
save_dir = "data/processed"
os.makedirs(save_dir, exist_ok=True)

for disease in disease_map.keys():

    print(f"\nCreating dataset for {disease}...")

    X, y = create_dataset(
        df=df,
        feature_waves=feature_waves,
        prediction_horizon=prediction_horizon,
        target=disease
    )

    # Drop person_id if present
    X = X.drop(columns=['person_id'], errors='ignore')

    # Ensure y is a Series
    if isinstance(y, pd.DataFrame):
        y = y.iloc[:, 0]

    target_name = f"incident_{disease}_{years}yr"

    dataset = pd.concat(
        [
            X.reset_index(drop=True),
            y.reset_index(drop=True).rename(target_name)
        ],
        axis=1
    )

    # File paths
    csv_path = os.path.join(save_dir, f"randhrs_{disease}_{years}yr.csv")
    parquet_path = os.path.join(save_dir, f"randhrs_{disease}_{years}yr.parquet")

    # Save CSV
    dataset.to_csv(csv_path, index=False)

    # Save Parquet
    dataset.to_parquet(parquet_path, index=False)

    print(
        f"{disease}: shape = {dataset.shape}, "
        f"positives = {dataset[target_name].sum()}, "
        f"saved to {csv_path} & {parquet_path}"
    )



Creating dataset for diabetes...


/tmp/ipykernel_124450/1075262581.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = pd.concat(all_data, ignore_index=True)


diabetes: shape = (98033, 134), positives = 6357.0, saved to data/processed/randhrs_diabetes_4yr.csv & data/processed/randhrs_diabetes_4yr.parquet

Creating dataset for cvd...


/tmp/ipykernel_124450/1075262581.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = pd.concat(all_data, ignore_index=True)


cvd: shape = (96166, 134), positives = 7888.0, saved to data/processed/randhrs_cvd_4yr.csv & data/processed/randhrs_cvd_4yr.parquet

Creating dataset for stroke...


...

cancer: shape = (105724, 134), positives = 4969.0, saved to data/processed/randhrs_cancer_4yr.csv & data/processed/randhrs_cancer_4yr.parquet

Creating dataset for hibp...


/tmp/ipykernel_124450/1075262581.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = pd.concat(all_data, ignore_index=True)


hibp: shape = (57089, 134), positives = 10084.0, saved to data/processed/randhrs_hibp_4yr.csv & data/processed/randhrs_hibp_4yr.parquet

Creating dataset for arthritis...


/tmp/ipykernel_124450/1075262581.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataset = pd.concat(all_data, ignore_index=True)


arthritis: shape = (55325, 134), positives = 9486.0, saved to data/processed/randhrs_arthritis_4yr.csv & data/processed/randhrs_arthritis_4yr.parquet
